# Matrix Product States

In [17]:
import numpy as np

## TT-SVD

In [18]:
N=10 # number of sites/spins
d=2 # physical dimension

eps=0 # SVD truncation error

tens = np.random.rand(*[d] * N) # high-dimensional tensor
A=[] # list for storing MPS tensors
tens.shape

(2, 2, 2, 2, 2, 2, 2, 2, 2, 2)

In [19]:
# Reshape (step 1)
tmp = tens.reshape(tens.shape[0], -1) # (temporary tensor)
tmp.shape

(2, 512)

In [20]:
# SVD + Truncate (step 2)
U, s, Vt = np.linalg.svd(tmp)

# Truncate singular values such that truncation error is less than or equal to eps
where_truncation_error_is_lower_than_eps = np.where(np.cumsum(s[::-1]**2) <= eps**2)[0]
num_sv_to_discard = 0 if len(where_truncation_error_is_lower_than_eps) == 0 else int(1 + where_truncation_error_is_lower_than_eps[-1])
r = max(1, len(s) - num_sv_to_discard) # new rank

In [21]:
# Reshape and truncate U matrix, store as first MPS site
A.append(U[:,:r].reshape(1, d, r))
A[0].shape

(1, 2, 2)

In [22]:
# Contract s and Vt (step 3)
tmp = np.diagflat(s[:r]) @ Vt[:r,:]
tmp.shape

(2, 512)

In [23]:
# Reshape (step 4)
tmp = tmp.reshape(r * tens.shape[1], -1)
tmp.shape
# Repeat steps 2-4, N-1 times

(4, 256)

In [24]:
from copy import copy
def tt_svd(tens: np.ndarray, eps: float = 10**-6, rank = 10**12) -> list:
    """
    Compress a tensor to a MPS/TT using the TT-SVD algorithm.

    Args:
        tens: The input tensor
        eps: Truncation error for each SVD
    Return:
        An MPS/TT as a list of order-3 tensors (dummy bonds are added to boundary tensors)
    """
    dims = tens.shape
    N = len(dims)
    tmp = copy(tens)
    A = []
    r_prev = 1
    for i in range(N-1):
        # Reshape (step 4)
        tmp = tmp.reshape(r_prev * dims[i], -1)
        
        # SVD + Truncate (step 2)
        U, s, Vt = np.linalg.svd(tmp)
        # Truncate singular values such that truncation error is less than or equal to eps
        where_truncation_error_is_lower_than_eps = np.where(np.cumsum(s[::-1]**2) <= eps**2)[0]
        num_sv_to_discard = 0 if len(where_truncation_error_is_lower_than_eps) == 0 else int(1 + where_truncation_error_is_lower_than_eps[-1])
        r = min(rank, max(1, len(s) - num_sv_to_discard)) # new rank
        
        # Reshape and truncate U matrix, store in return list
        A.append(U[:,:r].reshape(r_prev, dims[i], r))
        
        # Contract s and Vt (step 3)
        tmp = np.diagflat(s[:r]) @ Vt[:r,:]
        r_prev = r
    A.append(tmp.reshape(r_prev, dims[-1], 1))
    return A

In [70]:
eps=10**-1
mps = tt_svd(tens, eps=eps)

In [71]:
# Show bond dimensions/tt-ranks
def bdims(mps: list):
    return [site.shape[0] for site in mps] + [mps[-1].shape[-1]]

In [27]:
bdims(mps)

[1, 2, 4, 8, 16, 31, 16, 8, 4, 2, 1]

## Reconstructing full tensor 

In [28]:
def restore_full(mps: list) -> np.ndarray:
    """
    Restore full tensor from an MPS/TT

    Args:
        mps: List of order-3 tensors representing an MPS/TT

    Return:
        The full tensor
    """
    tmp = mps[0]
    dims = [site.shape[1] for site in mps]
    for site in mps[1:]:
        tmp = np.einsum('iuj,jvk->iuvk', tmp, site)
        tmp = tmp.reshape(tmp.shape[0], tmp.shape[1] * tmp.shape[2], tmp.shape[3])
    return tmp.reshape(dims)

In [29]:
# The TT/MPS approximation error
np.linalg.norm(tens - restore_full(mps)) 

0.020579719304961303

In [30]:
# Theoreterical upper bound for the TT/MPS approximation error (if each SVD truncation error is <= eps)
eps * np.sqrt(N)

0.316227766016838

In [31]:
# Number of parameters in MPS
sum(np.count_nonzero(site) for site in mps)

2664

In [32]:
# Number of parameters in orginal tensor
np.count_nonzero(tens)

1024

## Tensors with low TT-rank

The above random tensor doesn't have a low TT-rank and is not approximated well by a MPS/TT with low rank. Below is an example of a tensor which does have a low TT-rank (rank-2): a sinusoidal signal reshaped into a tensor. An even lower TT-rank tensor (rank-1) is that given by reshaping the exponential function.

In [33]:
N=10
d=2

w=1
phi=0.5
g=0.1
def x(t):
    return np.cos(w*t + phi)
    # return np.exp(-g*t)

tens = np.fromiter((x(t) for t in range(d**N)), dtype=np.float64).reshape([d] * N)
mps = tt_svd(tens, eps=10**-10)

In [34]:
bdims(mps)

[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]

In [35]:
np.linalg.norm(tens - restore_full(mps)) 

3.140316778841076e-14

In [36]:
sum(np.count_nonzero(site) for site in mps)

72

In [37]:
np.count_nonzero(tens)

1024

## Getting tensor elements in the MPS/TT format

In [65]:
from functools import reduce
def get_index(mps: list, inds):
    return reduce(np.matmul, (site[:, ind, :] for site, ind in zip(mps, inds)))[0,0]

In [66]:
get_index(mps, [0, 1, 1, 0, 0, 1, 0, 1, 1, 1])

0.6160760762757913

In [40]:
tens[0, 1, 1, 0, 0, 1, 0, 1, 1, 1]

0.6160760762757924

## Product States
Product state can be represented by MPS with bond dimension 1

In [57]:
def product_state_mps(state):
    """
    Generate an MPS representing a product state

    Args:
        state: A list of state vectors 
    """
    return [np.array(s).reshape(1, len(s), 1) for s in state]

In [58]:
mps_1 = product_state_mps([[1,0], [0,1], [1, 0], [0, 1]]) # |psi> = |01010>

In [59]:
bdims(mps_1)

[1, 1, 1, 1, 1]

In [67]:
# This mps has only 1 non-zero element:
get_index(mps_1, [0, 1, 0, 1])

1

In [61]:
sp = [1/np.sqrt(2), 1/np.sqrt(2)]
sn = [1/np.sqrt(2), -1/np.sqrt(2)]
mps_2 = product_state_mps([sp, sp, sp, sp])

In [69]:
bdims(mps_2)

[1, 1, 1, 1, 1]

In [68]:
# All elements of this mps are 0.25 (1/sqrt(2)^4)
get_index(mps_2, [1, 0, 1, 0]) # |psi> = |s+s+s+s+> (superposition of all computational basis states, |0000>, |0001>, |0010>,

0.24999999999999992

## Entangled States
Entangled states require bond dimensions greater than 1

In [63]:
# Let's construct an MPS for the wave-function: |psi> = 1/sqrt(2)(|0000000000> + |1111111111>)
# This is an entangled state since the wave-function cannot be seperated into a product of single-site terms
coefficient_tensor = np.zeros([2]*10)
coefficient_tensor[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] = 1/np.sqrt(2)
coefficient_tensor[1, 1, 1, 1, 1, 1, 1, 1, 1, 1] = 1/np.sqrt(2)
mps_3 = tt_svd(coefficient_tensor) 
bdims(mps_3)

[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]

In [55]:
get_index(mps_3, [0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

array([[0.]])

## Calculating Observables

In [48]:
def mps_inner_product(v, w): # See exercises/tn_contractions.ipynb
    """
    Calculate in the inner product <v,w> of two MPS/TT

    Returns:
        The value of the inner product
    """
    rho = np.identity(1)
    for vi, wi in zip(v, w):
        rho = np.einsum('ac,asb,csd->bd', rho, vi, wi)
    return rho[0,0]

In [49]:
mps_inner_product(mps, mps)

511.9069911445415

In [50]:
# Overlap of mps1 and mps2
mps_inner_product(mps_1, mps_2)

0.24999999999999992

In [51]:
# Norm of mps1
mps_inner_product(mps_1, mps_1)

1.0

In [52]:
def compute_one_site_expectation(psi: list, operator: np.ndarray, site_index: int):
    """
    Compute the expectation value of an operator acting on a single site of an MPS

    Args:
        psi: The MPS/TT
        operator: The operator to measure
        site_index: The site on which the operator acts

    Returns:
        The expectation value
    """
    rho = np.identity(1)
    physical_dimensions = [site.shape[1] for site in psi]
    ops = [np.identity(d) for d in physical_dimensions]
    ops[site_index] = operator
    for ai, op, ai in zip(psi, ops, psi):
        rho = np.einsum('ac,asb,st,ctd->bd', rho, ai, op, ai)
    return rho[0,0]

In [53]:
sx = np.array([[0, 1], [1, 0]])
sy = np.array([[0, -1j], [1j, 0]])
sz = np.array([[1, 0], [0, -1]])
compute_one_site_expectation(mps_1, sz, 2) # Compute the expectation of sz on site 2

1.0

In [54]:
compute_one_site_expectation(mps_2, sx, 2) 

0.9999999999999993